# Stage 1 Model Selection: Summary of Results

Model selection was conducted using a method of time series cross-validation called rolling forecast origin. To avoid leakage of future observations, the method incrementally moves the forecast origin forward in time and then makes a prediction. For each new fold we implemented a stride of seven days. We performed a two stage model selection procedure. In the first stage, we used an aggregate regional level time series to screen and identify the most promising candidate models for up to a 365 day forecast (27 folds).

This notebook generates the summary result tables and charts for stage 1 of model selection (screening)


In [ ]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
if cwd[-7:] != "results":
    mypath = './results/model_selection/temp/'
    TABLE_PATH = './paper/tables/'
    FIGURE_PATH = './paper/figures/'
    APPENDIX_PATH = './paper/appendix/'
else:
    mypath = './model_selection/temp/'
    TABLE_PATH = '../paper/tables/'
    FIGURE_PATH = '../paper/figures/'
    APPENDIX_PATH = '../paper/appendix/'

In [ ]:
result_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
results_mean = pd.DataFrame()
results_med = pd.DataFrame()
results_mean_std = pd.DataFrame()

## Point Estimate Results

In [ ]:
error_measures = ['smape', 'rmse', 'mase', 'coverage_80', 'coverage_95']

In [ ]:
for metric in error_measures:
    to_read = [filename for filename in result_files if metric in filename]
    model_names = [name[:name.index('_')] for name in to_read]
    
    for filename, model_name in zip(to_read, model_names):
        df = pd.read_csv(mypath + filename, index_col=0)

        prefix = model_name + '_' + metric
        results_mean[prefix + '_mean'] = df.mean()
        results_mean[prefix  + '_std'] = df.std()
        results_med[prefix + '_med'] = df.median()
        results_med[prefix + '_iqr'] = df.quantile(0.75) - df.quantile(0.25)
        
        results_mean_std[prefix] = results_mean[prefix + '_mean'].map('{:,.2f}'.format) \
            + ' (' + results_mean[prefix  + '_std'].map('{:,.2f}'.format) + ')'

In [ ]:
results_mean.filter(like="fbp")

## Formatted results

In [ ]:
point_forecasts = pd.concat([results_mean_std.filter(like='smape'), 
                             results_mean_std.filter(like='rmse'),
                             results_mean_std.filter(like='mase')], axis=1)

In [ ]:
pf_smape = point_forecasts.filter(like='smape')
pf_smape.reindex(sorted(pf_smape.columns), axis=1)

In [ ]:
pf_rmse = point_forecasts.filter(like='rmse')
pf_rmse.reindex(sorted(pf_rmse.columns), axis=1)

In [ ]:
pf_mase = point_forecasts.filter(like='mase')
pf_mase.reindex(sorted(pf_mase.columns), axis=1)

In [ ]:
columns = pf_rmse.columns
columns = [s.replace('_rmse', '') for s in columns]
pf_rmse.columns = columns
pf_rmse.sort_index(axis=1)

In [ ]:
columns = pf_smape.columns
columns = [s.replace('_smape', '') for s in columns]
pf_smape.columns = columns
pf_smape.sort_index(axis=1)

In [ ]:
columns = pf_mase.columns
columns = [s.replace('_mase', '') for s in columns]
pf_mase.columns = columns
pf_mase.sort_index(axis=1)

In [ ]:
coverage = results_mean_std.filter(like='80')
coverage

In [ ]:
coverage_80 = results_mean_std.filter(like='80')
columns = coverage_80.columns
columns = [s.replace('_coverage_80', '') for s in columns]
coverage_80.columns = columns
coverage_80.sort_index(axis=1)

In [ ]:
coverage_95 = results_mean_std.filter(like='95')
columns = coverage_95.columns
columns = [s.replace('_coverage_95', '') for s in columns]
coverage_95.columns = columns
coverage_95.sort_index(axis=1)

In [ ]:
#Write these rather large tables to a latex file to use in appendix

#Table S1: MASE
pf_mase.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}tableS1.tex')

#Table S2: 80% Coverage
coverage_80.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}tableS2.tex')

#Table S3: 95% coverage
coverage_95.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}tableS3.tex')

#extra rables for sMAPE and RMSE - not used in paper.
pf_smape.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}symmetricmape.tex')
pf_rmse.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}rootmeansqua.tex')


print(f'Appendix summary tables written to {APPENDIX_PATH}')

## Naive Benchmark (Seasonal Naive)

A naive baseline forecasting method was chosen. This was to ensure that the sophisticated methods we test in the study were only considered for the final benchmark if they provided more more accurate point forecasts than the simplest of models. As emergency care demand data are seasonal we opted for the well-known Seasonal Naive method.  This method works by using the most recent observation for the same day and carrying it forward.  For example, if we are forecasting next Tuesday then the observation from the most recent Tuesday is used as the predicted value.

In [ ]:
pf_snaive = point_forecasts.filter(like='snaive')
pf_snaive.reindex(sorted(pf_snaive.columns), axis=1)
columns = pf_snaive.columns
columns = [s.replace('_rmse', '') for s in columns]
columns = [s.replace('_smape', '') for s in columns]
columns = [s.replace('_mase', '') for s in columns]
pf_snaive.columns = ['sMAPE', 'RMSE', 'MASE']

In [ ]:
pf_snaive[['MASE', 'sMAPE', 'RMSE']].to_latex(buf=f'{TABLE_PATH}table2.tex')

In [ ]:
pf_snaive[:-1]

In [ ]:
print('\nTable 2: Cross-Validation of Seasonal Naive Point Forecasts')
print(pf_snaive[:-1])